In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.functions import current_date, lit

In [0]:
df_reservas_ota = (
    spark.read.table("production.raw.tb_reservas_ota")
    .select(
        F.col("reserva_ota_id"),
        F.col("reserva_id"),
        F.col("canal"),
        F.col("comissao_percent"),
        F.col("data_reserva")
    )
    .withColumn("data_reserva", F.to_date(F.col("data_reserva")))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_reservas_ota.limit(5))

In [0]:
catalog_table = "production.trusted.tb_reservas_ota"

if spark.catalog.tableExists(catalog_table):
    delta_table = DeltaTable.forName(spark, catalog_table)
    
    # Merge (upsert) com atualização da update_date
    delta_table.alias("target").merge(
        df_reservas_ota.alias("source"),
        "target.reserva_ota_id = source.reserva_ota_id"
    ).whenMatchedUpdate(
        set = {
            **{col: f"source.{col}" for col in df_reservas_ota.columns if col != "update_date"},
            "update_date": "current_date()"
        }
    ).whenNotMatchedInsertAll() \
     .execute()
else:
    # Se não existir, cria a tabela Delta incluindo a update_date
    (
        df_reservas_ota
        .withColumn("update_date", lit(""))
        .write
        .format("delta")
        .saveAsTable(catalog_table)
    )